<a href="https://colab.research.google.com/github/rachelyayra/AIAssignment/blob/main/SEMEVAL_TABULAR_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview of Code Implementation

This notebook demonstrates the implementation of a question-answering system over tabular data using the TAPAS and T5 models. The goal is to train a model that can effectively answer questions based on the context provided by structured tables.

## Key Components:

1. **Data Preparation**:
   - The dataset consists of tables that are structured for question-answering tasks.
   - The first 1,000 tables are selected for the training set, while the subsequent 300 tables are designated for validation.

2. **Model Initialization**:
   - Models are initialized using the appropriate architectures (TAPAS and T5) designed for processing tabular data and generating text-based responses.

3. **Training Loop**:
   - The training process includes:
     - Forward pass through the model with input data.
     - Loss computation using the CrossEntropyLoss function.
     - Backpropagation to optimize the model parameters through the AdamW optimizer.
     - Losses are collected for each batch to monitor the training progress.

4. **Validation**:
   - A validation phase is integrated to assess the model's performance on unseen data after each epoch, ensuring that the model generalizes well.

5. **Error Handling**:
   - The implementation includes error handling to gracefully skip batches that encounter issues during processing.

6. **Batch Loss Tracking**:
   - The notebook maintains a list of batch losses, which can be analyzed after training to understand the model's learning behavior over time.


# HOUSE KEEPING

In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import pandas as pd
from datasets import load_dataset
from transformers import TapasTokenizer, TapasForQuestionAnswering

import torch
import torch.nn as nn
from transformers import TapasModel, T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
from transformers import TapasModel, T5ForConditionalGeneration, T5Tokenizer, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer

# DATA

In [ ]:
# Load the QA pairs
semeval_dev_qa = load_dataset("cardiffnlp/databench", name="semeval", split="dev")

README.md:   0%|          | 0.00/43.6k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/65 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/49 [00:00<?, ?it/s]

qa.parquet:   0%|          | 0.00/8.47k [00:00<?, ?B/s]

qa.parquet:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

...

qa.parquet:   0%|          | 0.00/7.72k [00:00<?, ?B/s]

qa.parquet:   0%|          | 0.00/8.29k [00:00<?, ?B/s]

qa.parquet:   0%|          | 0.00/9.46k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/988 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/320 [00:00<?, ? examples/s]

In [ ]:
# Explore the data
print(semeval_dev_qa)


# Check the structure of the first dataset
print(semeval_dev_qa['dataset'][0])  # View the first QA pair

Dataset({
    features: ['question', 'answer', 'type', 'columns_used', 'column_types', 'sample_answer', 'dataset'],
    num_rows: 320
})
050_ING


In [ ]:
for i in semeval_dev_qa:
  print(f"Question: {i['question']}")
  print(f"Answer: {i['answer']}")
  print(f"Sample Answer: {i['sample_answer']}")
  print(f"Columns: {i['columns_used']}")
  print(f"Columns: {i['column_types']}")

Question: Is the most favorited author mainly communicating in Spanish?
Answer: True
Sample Answer: True
Columns: ['favorites', 'lang']
Columns: ['category', 'category']
Question: Does the author with the longest name post mainly original content?
Answer: True
Sample Answer: False
Columns: ['author_name', 'type']
Columns: ['category', 'category']
Question: Is there an author who received no retweets for any of their posts?
Answer: True
Sample Answer: True
Columns: ['author_name', 'retweets']
Columns: ['category', 'number[uint8]']
Question: Are there any posts that do not contain any links?
Answer: True
Sample Answer: True
Columns: ['links']
Columns: ['list[url]']
Question: How many unique authors are in the dataset?
Answer: 3765
Sample Answer: 20
Columns: ['author_name']
Columns: ['category']
Question: What is the length of the longest post (based on the number of words)?
Answer: 61
Sample Answer: 49
Columns: ['text']
Columns: ['text']
Question: What is the total number of retweets rec

# TRAINING

In [ ]:
# Load the TAPAS model and extract the encoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tapas_model = TapasModel.from_pretrained("google/tapas-base")
tapas_encoder = tapas_model.to(device)
tokenizer = AutoTokenizer.from_pretrained("google/tapas-base")
# Load the T5 model (decoder) and tokenizer
t5_model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)
t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Set the models to training mode
t5_model.train()

tokenizer_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
for param in tapas_encoder.parameters():
    param.requires_grad = False

In [ ]:
class TAPAST5Model(nn.Module):
    def __init__(self, tapas_model_name="google/tapas-base", t5_model_name="t5-base"):
        super(TAPAST5Model, self).__init__()

        # Load TAPAS encoder and T5 decoder
        self.tapas_encoder = tapas_encoder
        self.t5_decoder = t5_model

        # Get hidden sizes of TAPAS and T5
        self.tapas_hidden_size = self.tapas_encoder.config.hidden_size
        self.t5_hidden_size = self.t5_decoder.config.d_model

        # Linear layer to map TAPAS hidden size to T5 hidden size (if they differ)
        if self.tapas_hidden_size != self.t5_hidden_size:
            self.mapping_layer = nn.Linear(self.tapas_hidden_size, self.t5_hidden_size)
        else:
            self.mapping_layer = None  # No need for mapping if dimensions are already the same

    def forward(self, input_ids, attention_mask, decoder_input_ids, target_ids):
        # TAPAS encoder forward pass
        encoder_outputs = self.tapas_encoder(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = encoder_outputs.last_hidden_state  # Shape: (batch_size, seq_len, tapas_hidden_size)

        # If TAPAS and T5 have different hidden sizes, pass through the linear mapping layer
        if self.mapping_layer:
            hidden_states = self.mapping_layer(hidden_states)  # Shape: (batch_size, seq_len, t5_hidden_size)

        # T5 decoder forward pass
        outputs = self.t5_decoder(
            decoder_input_ids=decoder_input_ids,
            encoder_outputs=(hidden_states,),  # Pass encoder hidden states to T5 decoder
            labels=target_ids  # Use target sequence for teacher forcing
        )

        return outputs

    def generate_answer(self, input_ids, attention_mask):
        # Get encoder outputs
        encoder_outputs = self.tapas_encoder(input_ids=input_ids, attention_mask=attention_mask)

        # Generate answer using T5
        generated_ids = self.t5_decoder.generate(
            encoder_outputs=encoder_outputs,
            max_length=10
        )

        generated_answer = t5_tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        return generated_answer
# Example usage:
# Assuming you have input_ids, attention_mask, decoder_input_ids, and target_ids ready
model = TAPAST5Model()

In [ ]:
import os
import shutil
import pandas as pd
import torch
from transformers import  TapasForQuestionAnswering
from IPython.display import display, HTML


# Convert table to tokenizer-friendly format
def convert_table_to_inputs(table_data, queries):
    table = pd.DataFrame(table_data.values.tolist(), columns=table_data.columns)
    table = table.map(str)

    # Tokenize the inputs using the Tapas tokenizer
    inputs = tokenizer(
        table=table,
        queries=queries,
        padding="max_length",
        max_length = 512,
        truncation=True,
        return_tensors="pt"
    )

    inputs = {key: value.to(device) for key, value in inputs.items()}

    return inputs, table

In [ ]:
all_qa = load_dataset("cardiffnlp/databench", name="qa", split="train").to_pandas()

# Define a custom dataset
class TabularQADataset(Dataset):
    def __init__(self, qa_data):
        self.qa_data = qa_data

    def __len__(self):
        return len(self.qa_data)

    def __getitem__(self, idx):

        question = self.qa_data.loc[idx, 'question']
        dataset_id = self.qa_data.loc[idx, 'dataset']

        # Load the corresponding table
        df = pd.read_parquet(f"hf://datasets/cardiffnlp/databench/data/{dataset_id}/sample.parquet").head(250)

        # Convert the table to a list of lists or a format suitable for your model
        # Here, we concatenate all rows into a string or can directly use DataFrame for processing
        inputs, table = convert_table_to_inputs(df, str(question))


        # Prepare the answer (assuming the answer is in the 'answer' field)
        answer = self.qa_data.loc[idx, 'answer']
        if answer == None:
          answer = 'None'
        # Tokenize the answer using the T5 tokenizer
        print(f"Answer: {answer}")
        target = t5_tokenizer(answer, padding='max_length', truncation=True, return_tensors="pt", max_length=32)

        target = {key: value.to(device) for key, value in target.items()}
        decoder_input_ids = torch.cat([torch.tensor([[t5_tokenizer.pad_token_id]]).to(device), target['input_ids'][:, :-1]], dim=1)
        return inputs['input_ids'].squeeze(0), inputs['attention_mask'].squeeze(0), decoder_input_ids.squeeze(), target['input_ids'].squeeze().long(), target['attention_mask'].squeeze().long()   # Labels for the target sequence


# Create dataset and dataloader
dataset = TabularQADataset(all_qa[:1200].reset_index())
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)  # Adjust batch size as needed

val_dataset = TabularQADataset(all_qa[1000:].reset_index())
val_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

Resolving data files:   0%|          | 0/65 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
import torch
import torch.nn as nn
from torch.optim import AdamW

# Assuming 'model', 'train_dataloader', and 'val_dataloader' are defined.
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

# Training loop
num_epochs = 4
train_losses = []
val_losses = []
val_accuracies = []

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0  # Accumulate the total loss for the epoch
    num_batches = 0  # Keep track of the number of batches

    # Training phase
    model.train()  # Set model to training mode
    for batch_num, (input_ids, attention_mask, decoder_inputs, target_ids, at) in enumerate(dataloader):
        if batch_num == 10:
          break
        try:
            print(f"Batch {batch_num}: at:", input_ids.shape)
            print(f"Batch {batch_num}: Attention mask shape:", attention_mask.shape)

            # Clear gradients
            optimizer.zero_grad()

            # Forward pass
            output = model(input_ids, attention_mask, decoder_inputs, target_ids)

            # Compute loss
            logits = output.logits
            loss = loss_fn(logits.view(-1, logits.size(-1)), target_ids.view(-1))  # Correct the target_ids slicing

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Accumulate loss
            total_loss += loss.item()
            train_losses.append(loss.item())  # Save the loss for this batch
            num_batches += 1

            print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_num}], Loss: {loss.item():.4f}")

        except ValueError as e:
            print(f"ValueError encountered in Batch {batch_num}: {e}. Skipping this batch.")
            continue  # Skip the current batch and continue to the next one

    if num_batches > 0:
        avg_loss = total_loss / num_batches
        print(f"Epoch [{epoch + 1}/{num_epochs}] Average Training Loss: {avg_loss:.4f}")

    # Validation phase
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0.0
    val_num_batches = 0
    correct_predictions = 0

    with torch.no_grad():  # No gradients needed for validation
        for val_batch_num, (input_ids, attention_mask, decoder_inputs, target_ids, at) in enumerate(val_dataloader):
            try:
                # Forward pass
                output = model(input_ids, attention_mask, decoder_inputs, target_ids)

                # Compute validation loss
                logits = output.logits
                val_loss = loss_fn(logits.view(-1, logits.size(-1)), target_ids.view(-1))  # Correct the target_ids slicing

                # Accumulate validation loss
                total_val_loss += val_loss.item()
                val_num_batches += 1

                # Calculate accuracy
                predictions = torch.argmax(logits, dim=-1)
                correct_predictions += (predictions.view(-1) == target_ids.view(-1)).sum().item()

                print(f"Validation Batch [{val_batch_num}], Loss: {val_loss.item():.4f}")

            except ValueError as e:
                print(f"ValueError encountered in Validation Batch {val_batch_num}: {e}. Skipping this batch.")
                continue  # Skip the current validation batch

    if val_num_batches > 0:
        avg_val_loss = total_val_loss / val_num_batches
        val_accuracy = correct_predictions / (val_num_batches * target_ids.numel())  # Compute accuracy
        val_losses.append(avg_val_loss)  # Save validation loss
        val_accuracies.append(val_accuracy)  # Save validation accuracy
        print(f"Epoch [{epoch + 1}/{num_epochs}] Average Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

# After training, you can print or analyze the collected losses
print("All training batch losses:", train_losses)
print("All validation losses:", val_losses)
print("All validation accuracies:", val_accuracies)


/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


Answer: ['Private', 'children']


/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


Answer: [0.9794365922809228, 0.9723660656030668, 0.954299437125917, 0.9362989453985364, 0.9307917067583288]


/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


...

Answer: [2, 1, 6, 0, 7, 8]


/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


Answer: True


/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)


KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer, TapasForQuestionAnswering
import pandas as pd

import torch

# Set the model to evaluation mode
# Load your custom model
model.eval()  # Set the model to evaluation mode

# Prepare the input data
table_data = pd.read_parquet(f"hf://datasets/cardiffnlp/databench/data/{semeval_dev_qa['dataset'][1]}/sample.parquet").head(250)
print(table_data)

queries = ['Is the most favorited author mainly communicating in Spanish?']

input_ids, table = convert_table_to_inputs(table_data, queries)

with torch.no_grad():
    generated_ids = model.generate_answer(input_ids['input_ids'], input_ids['attention_mask'])


# Output result
print("Question:", queries)
print("Generated Answer:", generated_ids)


                     id            author_id               author_name  \
0   1166109737643139072           1007329988                Lau_Arbona   
1   1166096471650963456   868749503461040130             Jimmy McNulty   
2   1166078033691840513           2882602743                  la chula   
3   1166068194299252736           1348393310                  Mr_Gosky   
4   1166066536093814786            867101041                 Angeloclv   
5   1166056669689896960            133189676                       Eva   
6   1166055519024553989            332723994              𝕻𝖆𝖇𝖑𝖔 𝕬𝖗𝖗𝖔𝖞𝖔   
7   1166023559250157570           1072994982                   xAnubis   
8   1165996371750653952            295455234         Unnamed player  💬   
9   1165995541303582720             14524588              Sergio Gómez   
10  1165976106161385474  1094963080948260864                     Astur   
11  1165972146729947137  1144999258023747584                  Don Vito   
12  1165963665725673472            139

/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


Question: ['Is the most favorited author mainly communicating in Spanish?']
Generated Answer: 


## The model Fails to generate an answer

In [ ]:
def generate_predictions(qa_dataset, max_rows=100):
    predictions = []

    for idx, row in enumerate(qa_dataset):
        print(f"Row Number: {idx}")
        question = row['question']
        table_key = row['dataset']  # Get the dataset associated with the question

        df = pd.read_parquet(f"hf://datasets/cardiffnlp/databench/data/{table_key}/all.parquet").head(250)
        queries = [question]

        input_ids, table = convert_table_to_inputs(df, queries)

        # Get TAPAS encoder outputs
        with torch.no_grad():  # Disable gradient tracking for evaluation
            encoder_outputs = tapas_encoder(input_ids=input_ids['input_ids'], attention_mask=input_ids['attention_mask'])

        # Get hidden states
        hidden_states = encoder_outputs.last_hidden_state

        # Generate predictions using T5's generate method
        with torch.no_grad():
            generated_ids = t5_model.generate(
                encoder_outputs=encoder_outputs,
                max_length=32,  # Adjust max length according to your task
                num_beams=5,  # You can use beam search or set this to 1 for greedy decoding
                early_stopping=True
            )

        # Decode the generated token IDs back to text
        predicted_text = t5_tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        print(f"Predicted Text: {predicted_text}")

        predictions.append(predicted_text)
        break

    return predictions


In [ ]:
def generate_predictions_sample(qa_dataset, max_rows=100):
    predictions = []

    for idx, row in enumerate(qa_dataset):
        print(f"Row Number: {idx}")
        question = row['question']
        table_key = row['dataset']  # Get the dataset associated with the question

        df = pd.read_parquet(f"hf://datasets/cardiffnlp/databench/data/{table_key}/sample.parquet").head(250)
        queries = [question]

        input_ids, table = convert_table_to_inputs(df, queries)

        # Get TAPAS encoder outputs
        with torch.no_grad():  # Disable gradient tracking for evaluation
            encoder_outputs = tapas_encoder(input_ids=input_ids['input_ids'], attention_mask=input_ids['attention_mask'])

        # Get hidden states
        hidden_states = encoder_outputs.last_hidden_state

        # Generate predictions using T5's generate method
        with torch.no_grad():
            generated_ids = t5_model.generate(
                encoder_outputs=encoder_outputs,
                max_length=32,  # Adjust max length according to your task
                num_beams=5,  # You can use beam search or set this to 1 for greedy decoding
                early_stopping=True
            )

        # Decode the generated token IDs back to text
        predicted_text = t5_tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        print(f"Predicted Text: {predicted_text}")

        predictions.append(predicted_text)
        break

    return predictions

### SAVE FILES

In [ ]:
predictions = generate_predictions(semeval_dev_qa)

In [ ]:
predictions_lite = generate_predictions_sample(semeval_dev_qa)

In [ ]:
with open('predictions_lite.txt', 'w') as f:
  for prediction in predictions_lite:
    f.write(f"{prediction}\n")  # Write each prediction on a new line

In [ ]:
with open('predictions.txt', 'w') as f:
  for prediction in predictions:
    f.write(f"{prediction}\n")  # Write each prediction on a new line
